# Traning RNN on Simulation

### Imports

In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

D:\Programme\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



2.15.0


In [276]:
import random
import math
import pickle
import Simulation.KNNSim as sim

### Load data and helper tables

In [3]:
startIndexes = pickle.load(open('Simulation/E0F1_index_transmision.pkl', 'rb'))
usablePVindextable = pickle.load(open('Simulation/E0F1_index_I1.pkl', 'rb'))
pvNames = pickle.load(open('Simulation/E0F1Names.pkl', 'rb'))

### Define the shape of the input...

In [362]:
input_dim =  len(usablePVindextable)*2 + 1 # permissionbools for PVs (dependand on subsection),current PV values , current current value
output_dim = len(usablePVindextable)
input_dim

25

### Define the model

In [363]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True, stateful=True, batch_input_shape=(1,None,input_dim)),
    tf.keras.layers.LSTM(32, return_sequences=True, stateful=True, batch_input_shape=(1,None,input_dim)),
    tf.keras.layers.LSTM(32, return_sequences=False, stateful=True),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(output_dim, activation='linear') # Adjust based on your output requirements
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.build()
model.compile()

In [364]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_45 (LSTM)              (1, None, 32)             7424      
                                                                 
 lstm_46 (LSTM)              (1, 16)                   3136      
                                                                 
 dense_54 (Dense)            (1, 64)                   1088      
                                                                 
 dense_55 (Dense)            (1, 32)                   2080      
                                                                 
 dense_56 (Dense)            (1, 16)                   528       
                                                                 
 dense_57 (Dense)            (1, 16)                   272       
                                                                 
 dense_58 (Dense)            (1, 12)                 

### Helper functions

In [312]:
def generateRandomBoolArr(length, numOfOnes):
    result = np.zeros(length, dtype=int)
    indices = np.random.choice(length, numOfOnes, replace=False)
    result[indices] = 1
    return result

In [313]:
def modelOutToPosition(modelOut, allowedPVs, oldPosition):
    for i, change in enumerate(modelOut):
        if allowedPVs[i]:
            oldPosition[usablePVindextable[i]] += change
        modelOut[i] = oldPosition[usablePVindextable[i]]
    return [oldPosition, modelOut]

In [314]:
def biasedRandomNumber(max_value):
    prob = np.arange(1, max_value + 1)
    # Normalize probabilities to sum to 1
    prob = prob / np.sum(prob)
    result = max_value + 1 - np.random.choice(np.arange(1, max_value + 1), p=prob)
    return result

In [311]:
#40104 is the index for the global optimum
optimum = list(sim.states[40104][0])
baseOptimum = []

for i in usablePVindextable:
    baseOptimum.append(optimum[i])

var_I1 = [0.4656249999999993,0.17000000000000035,0.6524999999999999,0.14566250000000042,0.34001250000000066,0.19999999999999996,0.14499999999999957,0.13628749999999912,2.239959716796875,1.40057373046875,2.3402099609375,1.9005126953125]

def get_StartingPossition():
    sectionPos = []
    for i, variance in enumerate(var_I1):
        newValue = baseOptimum[i] + (random.random()-0.5) * variance
        optimum[usablePVindextable[i]] = newValue
        sectionPos.append(newValue)
    return [np.array(optimum), np.array(sectionPos)]

count = 0
for i in range(1000):
    get_StartingPossition()
    if sim.get(optimum) < 0:
        count += 1
print(count/1000)

0.529


## Trainig Loop

In [ ]:
# Training loop
runsPerEpoch = 100
epochs = 4000

bestLoss = 0
bestDec = 0

num_steps = 4.9
print("Starting Traning...")
for epoch in range(epochs):
    
    num_steps = min(num_steps + 0.01, 30)
    sumList = [0,0,0]
    for run in range(runsPerEpoch):
        # This is one run
        
        # Reset the LSTM states
        model.layers[0].reset_states()
        model.layers[1].reset_states() 
        
        # Choose a starting position
        position, sectionPos = get_StartingPossition()
        
        # Choose which PVs we can manipulate in this run
        allowedPVs = generateRandomBoolArr(output_dim, 12)
        
        # Init loss
        y = sim.get(position)
        minY = y
        startY = y
        
        states = []
        returns = []
        action = []
        fail = 0
        with tf.GradientTape() as tape:
            # Forward pass
            for step in range(math.floor(num_steps)):

                modelIn = np.reshape(np.concatenate((allowedPVs, sectionPos, [y]), axis=0), (1, 1, input_dim))
                modelOut = model(modelIn)
                action.append(modelOut)
                # We now have our next step
                position , sectionPos = modelOutToPosition(modelOut.numpy()[0], allowedPVs, position)
                # We can calculate the loss right here
                y = sim.get(position)
                returns.append((y - startY)/math.floor(num_steps))
                
                
                if y < minY: 
                    minY = y
            #We made a run Now to calculate a gradient
            bestImp = returns[-1]
            i = -1
            while i >= -len(returns):
                if returns[i] < bestImp:
                    bestImp = returns[i]
                returns[i] = bestImp
                i -= 1
            
            result = [(action * allowedPVs/ tf.reduce_sum(abs(action * allowedPVs))) * returns for action, returns in zip(action, returns)]
            loss = tf.reduce_sum(result) 

        # Backward pass
        gradients = tape.gradient(tf.constant(loss), model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))   
        sumList[0] += loss
        sumList[1] += minY - startY
        if startY != minY:
            sumList[2] += 1

    # Print epoch information
    print(f'Epoch {epoch+1}/{epochs}, AVG Loss: {sumList[0]/runsPerEpoch: .3f}, AVG improvment: {sumList[1]/runsPerEpoch: .3f}, Acc: {(sumList[2]/runsPerEpoch)*100: .1f}%')
    
    if sumList[0]/runsPerEpoch < bestLoss:
        bestLoss = sumList[0]/runsPerEpoch
        print(f'------------------------ New best Loss with {bestLoss: .3f} ------------------------')
        model.save('BestLoss.keras')
    
    if (sumList[1]/runsPerEpoch) * (sumList[2]/runsPerEpoch) < bestDec:
        bestDec = (sumList[1]/runsPerEpoch) * (sumList[2]/runsPerEpoch)
        print(f'------------------------ New best Dec with {bestDec: .3f} ------------------------')
        model.save('BestDec.keras')
    
    if epoch % 100 == 0:
        name = f'ModelE{epoch}_L{sumList[0]/runsPerEpoch: .2f}_I{sumList[1]/runsPerEpoch: .2f}_P{sumList[2]/runsPerEpoch: .2f}'
        print("Modelsnapshot saved as:" + name)
        model.save(name + '.keras')
        

Starting Traning...
Epoch 1/4000, AVG Loss:  1322.195, AVG improvment: -15.124, Acc:  39.0%
------------------------ New best Dec with -5.898 ------------------------
Modelsnapshot saved as:ModelE0_L 1322.19_I-15.12_P 0.39
Epoch 2/4000, AVG Loss:  1311.183, AVG improvment: -21.596, Acc:  38.0%
------------------------ New best Dec with -8.206 ------------------------
Epoch 3/4000, AVG Loss:  1327.549, AVG improvment: -15.850, Acc:  33.0%
Epoch 4/4000, AVG Loss:  1281.428, AVG improvment: -15.098, Acc:  33.0%
Epoch 5/4000, AVG Loss:  1301.931, AVG improvment: -14.069, Acc:  33.0%
Epoch 6/4000, AVG Loss:  1301.638, AVG improvment: -16.770, Acc:  37.0%
Epoch 7/4000, AVG Loss:  1292.627, AVG improvment: -15.758, Acc:  36.0%
Epoch 8/4000, AVG Loss:  1254.492, AVG improvment: -24.511, Acc:  47.0%
------------------------ New best Dec with -11.520 ------------------------
Epoch 9/4000, AVG Loss:  1279.883, AVG improvment: -16.272, Acc:  39.0%
Epoch 10/4000, AVG Loss:  1292.909, AVG improvment

Epoch 109/4000, AVG Loss:  1605.713, AVG improvment: -18.087, Acc:  35.0%
Epoch 110/4000, AVG Loss:  1652.778, AVG improvment: -15.068, Acc:  35.0%
Epoch 111/4000, AVG Loss:  1998.492, AVG improvment: -10.949, Acc:  37.0%
Epoch 112/4000, AVG Loss:  2003.438, AVG improvment: -9.137, Acc:  29.0%
Epoch 113/4000, AVG Loss:  1970.656, AVG improvment: -14.134, Acc:  35.0%
Epoch 114/4000, AVG Loss:  1991.078, AVG improvment: -14.179, Acc:  34.0%
Epoch 115/4000, AVG Loss:  1986.205, AVG improvment: -15.892, Acc:  35.0%
Epoch 116/4000, AVG Loss:  1952.705, AVG improvment: -19.886, Acc:  36.0%
Epoch 117/4000, AVG Loss:  1943.685, AVG improvment: -14.041, Acc:  33.0%
Epoch 118/4000, AVG Loss:  1963.679, AVG improvment: -15.906, Acc:  42.0%
Epoch 119/4000, AVG Loss:  1968.895, AVG improvment: -24.169, Acc:  40.0%
Epoch 120/4000, AVG Loss:  1975.219, AVG improvment: -18.908, Acc:  40.0%
Epoch 121/4000, AVG Loss:  1925.873, AVG improvment: -21.600, Acc:  43.0%
Epoch 122/4000, AVG Loss:  1901.168, AV

Epoch 216/4000, AVG Loss:  2312.363, AVG improvment: -29.820, Acc:  44.0%
Epoch 217/4000, AVG Loss:  2277.495, AVG improvment: -25.365, Acc:  46.0%
Epoch 218/4000, AVG Loss:  2211.311, AVG improvment: -32.792, Acc:  57.0%
Epoch 219/4000, AVG Loss:  2295.743, AVG improvment: -21.874, Acc:  40.0%
Epoch 220/4000, AVG Loss:  2281.089, AVG improvment: -25.094, Acc:  50.0%
Epoch 221/4000, AVG Loss:  2287.825, AVG improvment: -28.806, Acc:  56.0%
Epoch 222/4000, AVG Loss:  2297.965, AVG improvment: -24.084, Acc:  45.0%
Epoch 223/4000, AVG Loss:  2275.801, AVG improvment: -29.414, Acc:  49.0%
Epoch 224/4000, AVG Loss:  2327.933, AVG improvment: -28.720, Acc:  47.0%
Epoch 225/4000, AVG Loss:  2252.708, AVG improvment: -35.301, Acc:  53.0%
Epoch 226/4000, AVG Loss:  2233.275, AVG improvment: -38.117, Acc:  51.0%
Epoch 227/4000, AVG Loss:  2360.724, AVG improvment: -24.135, Acc:  44.0%
Epoch 228/4000, AVG Loss:  2252.223, AVG improvment: -40.256, Acc:  56.0%
------------------------ New best Dec 

Epoch 324/4000, AVG Loss:  2630.435, AVG improvment: -17.290, Acc:  43.0%
Epoch 325/4000, AVG Loss:  2562.210, AVG improvment: -32.723, Acc:  44.0%
Epoch 326/4000, AVG Loss:  2613.784, AVG improvment: -24.547, Acc:  49.0%
Epoch 327/4000, AVG Loss:  2588.847, AVG improvment: -37.227, Acc:  53.0%
Epoch 328/4000, AVG Loss:  2715.377, AVG improvment: -24.911, Acc:  46.0%
Epoch 329/4000, AVG Loss:  2616.463, AVG improvment: -22.604, Acc:  54.0%
Epoch 330/4000, AVG Loss:  2562.646, AVG improvment: -40.538, Acc:  59.0%
Epoch 331/4000, AVG Loss:  2667.918, AVG improvment: -30.944, Acc:  51.0%
Epoch 332/4000, AVG Loss:  2677.742, AVG improvment: -28.296, Acc:  54.0%
Epoch 333/4000, AVG Loss:  2638.477, AVG improvment: -26.330, Acc:  48.0%
Epoch 334/4000, AVG Loss:  2602.962, AVG improvment: -35.368, Acc:  52.0%
Epoch 335/4000, AVG Loss:  2611.563, AVG improvment: -20.992, Acc:  52.0%
Epoch 336/4000, AVG Loss:  2640.423, AVG improvment: -33.523, Acc:  57.0%
Epoch 337/4000, AVG Loss:  2588.928, A

Epoch 434/4000, AVG Loss:  3073.853, AVG improvment: -32.373, Acc:  54.0%
Epoch 435/4000, AVG Loss:  3033.259, AVG improvment: -35.249, Acc:  51.0%
Epoch 436/4000, AVG Loss:  3044.028, AVG improvment: -30.522, Acc:  49.0%
Epoch 437/4000, AVG Loss:  3106.158, AVG improvment: -25.900, Acc:  40.0%
Epoch 438/4000, AVG Loss:  3077.650, AVG improvment: -36.923, Acc:  51.0%
Epoch 439/4000, AVG Loss:  3138.132, AVG improvment: -25.324, Acc:  44.0%
Epoch 440/4000, AVG Loss:  3101.071, AVG improvment: -30.008, Acc:  43.0%
Epoch 441/4000, AVG Loss:  3055.029, AVG improvment: -30.572, Acc:  47.0%
Epoch 442/4000, AVG Loss:  3063.696, AVG improvment: -29.321, Acc:  49.0%
Epoch 443/4000, AVG Loss:  3015.260, AVG improvment: -25.925, Acc:  54.0%
Epoch 444/4000, AVG Loss:  3025.084, AVG improvment: -29.853, Acc:  41.0%
Epoch 445/4000, AVG Loss:  3023.127, AVG improvment: -26.692, Acc:  52.0%
Epoch 446/4000, AVG Loss:  3035.189, AVG improvment: -27.054, Acc:  49.0%
Epoch 447/4000, AVG Loss:  3069.919, A

...

Epoch 1520/4000, AVG Loss:  7267.466, AVG improvment: -36.231, Acc:  39.0%
Epoch 1521/4000, AVG Loss:  7315.690, AVG improvment: -23.988, Acc:  28.0%
Epoch 1522/4000, AVG Loss:  7304.069, AVG improvment: -26.673, Acc:  33.0%
Epoch 1523/4000, AVG Loss:  7311.689, AVG improvment: -23.461, Acc:  34.0%
Epoch 1524/4000, AVG Loss:  7260.408, AVG improvment: -27.430, Acc:  37.0%
Epoch 1525/4000, AVG Loss:  7238.524, AVG improvment: -26.330, Acc:  41.0%
Epoch 1526/4000, AVG Loss:  7221.980, AVG improvment: -35.600, Acc:  44.0%
Epoch 1527/4000, AVG Loss:  7256.334, AVG improvment: -28.186, Acc:  41.0%
Epoch 1528/4000, AVG Loss:  7269.340, AVG improvment: -22.586, Acc:  36.0%
Epoch 1529/4000, AVG Loss:  7278.482, AVG improvment: -26.573, Acc:  40.0%
Epoch 1530/4000, AVG Loss:  7242.415, AVG improvment: -36.446, Acc:  40.0%
Epoch 1531/4000, AVG Loss:  7264.101, AVG improvment: -33.138, Acc:  44.0%
Epoch 1532/4000, AVG Loss:  7221.116, AVG improvment: -42.213, Acc:  48.0%
Epoch 1533/4000, AVG Loss

Epoch 1629/4000, AVG Loss:  7739.638, AVG improvment: -21.927, Acc:  30.0%
Epoch 1630/4000, AVG Loss:  7765.004, AVG improvment: -16.248, Acc:  21.0%
Epoch 1631/4000, AVG Loss:  7759.991, AVG improvment: -12.657, Acc:  19.0%
Epoch 1632/4000, AVG Loss:  7743.401, AVG improvment: -21.627, Acc:  26.0%
Epoch 1633/4000, AVG Loss:  7759.129, AVG improvment: -16.413, Acc:  25.0%
Epoch 1634/4000, AVG Loss:  7752.773, AVG improvment: -17.067, Acc:  27.0%
Epoch 1635/4000, AVG Loss:  7757.867, AVG improvment: -16.458, Acc:  30.0%
Epoch 1636/4000, AVG Loss:  7757.626, AVG improvment: -20.944, Acc:  27.0%
Epoch 1637/4000, AVG Loss:  7737.994, AVG improvment: -35.072, Acc:  35.0%
Epoch 1638/4000, AVG Loss:  7760.080, AVG improvment: -23.665, Acc:  26.0%
Epoch 1639/4000, AVG Loss:  7753.335, AVG improvment: -15.203, Acc:  23.0%
Epoch 1640/4000, AVG Loss:  7763.646, AVG improvment: -22.426, Acc:  27.0%
Epoch 1641/4000, AVG Loss:  7755.726, AVG improvment: -20.852, Acc:  25.0%
Epoch 1642/4000, AVG Loss

Epoch 1738/4000, AVG Loss:  8064.559, AVG improvment: -29.663, Acc:  39.0%
Epoch 1739/4000, AVG Loss:  8111.040, AVG improvment: -19.645, Acc:  26.0%
Epoch 1740/4000, AVG Loss:  8086.638, AVG improvment: -17.223, Acc:  27.0%
Epoch 1741/4000, AVG Loss:  8092.053, AVG improvment: -17.971, Acc:  28.0%
Epoch 1742/4000, AVG Loss:  8058.464, AVG improvment: -31.301, Acc:  41.0%
Epoch 1743/4000, AVG Loss:  8032.798, AVG improvment: -25.555, Acc:  42.0%
Epoch 1744/4000, AVG Loss:  8054.822, AVG improvment: -26.478, Acc:  39.0%
Epoch 1745/4000, AVG Loss:  8051.294, AVG improvment: -20.876, Acc:  37.0%
Epoch 1746/4000, AVG Loss:  8013.172, AVG improvment: -21.801, Acc:  40.0%
Epoch 1747/4000, AVG Loss:  8041.301, AVG improvment: -19.525, Acc:  38.0%
Epoch 1748/4000, AVG Loss:  8020.076, AVG improvment: -22.983, Acc:  37.0%
Epoch 1749/4000, AVG Loss:  8040.518, AVG improvment: -18.245, Acc:  41.0%
Epoch 1750/4000, AVG Loss:  7926.796, AVG improvment: -36.013, Acc:  51.0%
Epoch 1751/4000, AVG Loss

Epoch 1847/4000, AVG Loss:  8498.960, AVG improvment: -14.514, Acc:  21.0%
Epoch 1848/4000, AVG Loss:  8490.112, AVG improvment: -21.337, Acc:  20.0%
Epoch 1849/4000, AVG Loss:  8514.339, AVG improvment: -12.908, Acc:  19.0%
Epoch 1850/4000, AVG Loss:  8485.161, AVG improvment: -27.600, Acc:  27.0%
Epoch 1851/4000, AVG Loss:  8499.060, AVG improvment: -17.399, Acc:  28.0%
Epoch 1852/4000, AVG Loss:  8517.392, AVG improvment: -11.714, Acc:  21.0%
Epoch 1853/4000, AVG Loss:  8521.613, AVG improvment: -12.192, Acc:  17.0%
Epoch 1854/4000, AVG Loss:  8525.843, AVG improvment: -14.776, Acc:  16.0%
Epoch 1855/4000, AVG Loss:  8515.824, AVG improvment: -11.735, Acc:  19.0%
Epoch 1856/4000, AVG Loss:  8522.957, AVG improvment: -13.157, Acc:  19.0%
Epoch 1857/4000, AVG Loss:  8517.864, AVG improvment: -11.681, Acc:  15.0%
Epoch 1858/4000, AVG Loss:  8512.863, AVG improvment: -19.195, Acc:  24.0%
Epoch 1859/4000, AVG Loss:  8503.457, AVG improvment: -18.683, Acc:  20.0%
Epoch 1860/4000, AVG Loss

Epoch 1956/4000, AVG Loss:  8907.114, AVG improvment: -9.190, Acc:  13.0%
Epoch 1957/4000, AVG Loss:  8875.663, AVG improvment: -19.857, Acc:  22.0%
Epoch 1958/4000, AVG Loss:  8901.256, AVG improvment: -10.015, Acc:  15.0%
Epoch 1959/4000, AVG Loss:  8889.166, AVG improvment: -12.034, Acc:  18.0%
Epoch 1960/4000, AVG Loss:  8896.947, AVG improvment: -19.124, Acc:  22.0%
Epoch 1961/4000, AVG Loss:  8892.193, AVG improvment: -19.105, Acc:  24.0%
Epoch 1962/4000, AVG Loss:  8892.149, AVG improvment: -7.454, Acc:  15.0%
Epoch 1963/4000, AVG Loss:  8894.605, AVG improvment: -12.343, Acc:  20.0%
Epoch 1964/4000, AVG Loss:  8899.023, AVG improvment: -10.151, Acc:  16.0%
Epoch 1965/4000, AVG Loss:  8887.880, AVG improvment: -18.532, Acc:  24.0%
Epoch 1966/4000, AVG Loss:  8892.125, AVG improvment: -7.478, Acc:  18.0%
Epoch 1967/4000, AVG Loss:  8878.202, AVG improvment: -15.761, Acc:  24.0%
Epoch 1968/4000, AVG Loss:  8860.827, AVG improvment: -13.371, Acc:  24.0%
Epoch 1969/4000, AVG Loss:  